## import libraries

In [2]:
import asyncio
import math
from math import log, sin, cos, tan, exp, sqrt, pi
import time
from random import randrange
import torch
import numpy as np
from classroom import Classroom
from classroom import Student
from classroom import BytesDataset
from classroom import GutenbergBytesDataset
from classroom import GutenbergBitsDataset
from classroom import GutenbergGPT2Dataset
from classroom import MLPLM, MyLM, ABPCNLM
from classroom import TransformerLM
from classroom import AdamW
from classroom import Sonny
from classroom import Floyd
from classroom import Plot
from classroom import Fun
from classroom import Count
from classroom import Sum
from classroom import Diff
from classroom import Log2Sum
from classroom import KalmanFilter1D
from classroom import MedianFilter
from classroom import TwoWindowFilter
from classroom import numel
from classroom import utf8decode, utf8encode, gpt2decode, gpt2encode
from classroom import utf8bitsdecode, utf8bitsencode
from pathlib import Path
import numba

Loading BokehJS ...

## initialize model

In [3]:
if True:
    path = '2021-11-27-0429.pt'

In [ ]:
if True:
    model = torch.load(path).to('cuda')

In [ ]:
if False:
    model = (
        MLPLM(
            n_vocab_in=50257,
            n_ctx=16,
            d_model=16,
            d_hidden=16,
            nonlinearity="GELU",
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        MyLM(
            n_vocab_in=50257,
            n_ctx=65,
            d_model=64,
            n_layers=1,
            d_hidden=256,
            nonlinearity="GELU",
            p_dropout=0.00,
            n_vocab_out=50257).to('cuda'))

In [ ]:
if False:
    model = (
        ABPCNLM(
            n_vocab_in=50257,
            n_ctx=128,
            d_model=8,
            n_layers=1,
            d_hidden=1024,
            nonlinearity="GELU",
            p_dropout=0.0,
            n_vocab_out=50257).to('cuda'))

In [6]:
if True:
    model = (
        TransformerLM(
            n_vocab_in=50257,
            n_vocab_out=50257,
            n_ctx=128,
            d_model=2**12,
            d_k=2**6,
            d_v=2**6,
            n_heads=2**6,
            d_hidden=2**12,
            n_layers=8,
            p_dropout_embedding=0,
            p_dropout_attn_mat=0,
            p_dropout_attn_out=0,
            p_dropout_mlp=0).to('cpu'))

In [7]:
numel(model)

1217881169

## initialize student

In [8]:
optimizer = AdamW(parameters=model.named_parameters())
dataset = GutenbergGPT2Dataset(device='cpu')
#dataset = GutenbergBytesDataset()

batch_size = None
example_length = model.n_ctx + 1

student= Student(
    model=model,
    optimizer=optimizer,
    dataset=dataset,
    batch_size=batch_size,
    example_length=example_length,
    device='cpu')

## schedule hyperparameters

In [9]:
student.batch_size = 1
student.example_length = 129
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    batch_multiplier = 1
    lr_base = 1e-5
    warm_up = 0
    lr = lambda n: 0 if n < warm_up else lr_base *(n%1000)/1000 
    student.optimizer.state[pn]["lr"]           = lambda n: lr(n)
    student.optimizer.state[pn]["beta1"]        = lambda n: 0.9
    student.optimizer.state[pn]["beta2"]        = lambda n: 0.999
    student.optimizer.state[pn]["weight_decay"] = lambda n: 0.001
    student.optimizer.state[pn]["update"]       = lambda n: (n < warm_up) or (n%batch_multiplier == 0)

## test a single iteration

In [ ]:
# model.language_model.crossentropyloss.crossentropyloss = torch.nn.CrossEntropyLoss(reduction='none')

In [10]:
student.study()

## initialize baseline

In [11]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)
    t_start = time.time()
    t_of_last_baseline = 0

## start training

In [12]:
import asyncio
async def train(student):
    while True:
        student.study()
        await asyncio.sleep(1e-4)

In [13]:
training_task = asyncio.create_task(train(student))

In [14]:
training_task

<Task pending name='Task-3' coro=<train() running at /tmp/ipykernel_428773/1424160439.py:5> wait_for=<Future finished result=None>>

## autocomplete

In [50]:
def autocomplete(model, prompt=None, n_generate=512,
                     n_ctx=None, temp=1.0,
                     encode=None, decode=None, output=None):
    Categorical = torch.distributions.Categorical
    if n_ctx is None:
        n_ctx = model.n_ctx
    if encode is None:
        encode = gpt2encode
        #encode = utf8encode
    if decode is None:
        decode = gpt2decode
        #decode = utf8decode
    if prompt is None:
        prompt = decode(student.dataset.batch(1, 2*n_ctx, offset=None).tolist()[0])  # kludge
    x = encode(prompt)
    x = x[-n_ctx:]
    prompt = decode(x)
    print(f"=== Prompt ===\n{prompt}\n=== Autocompletion ===\n")

    def sampler(x):
        x = list(x)
        for _ in range(n_generate):
            probs = model.inference(torch.tensor(x, dtype=torch.long, device="cuda").unsqueeze(0)).view(-1)[-model.n_vocab_out:]
            if temp > 0:
                y = Categorical(probs=probs**(1.0/temp)).sample().item()
            else:
                y = torch.argmax(probs).item()
            x = (x + [y])[-n_ctx:]
            if output is not None:
                output.append(y)
            yield y
    result = decode(list(sampler(x)))
    print(result)


When first the opposition of fact and ideal grows fully visible, a
spirit of fiery revolt, of fierce hatred of the gods, seems necessary
to the assertion of freedom. To defy with Promethean constancy a hostile
universe, to keep its evil always in view, always actively hated, to
refuse no pain that the malice of Power can invent, appears to be the
duty of all who will not bow before the inevitable. But indignation is
still a bondage, for it compels our thoughts to be occupied with an evil
world; and in the fierceness of desire from which rebellion springs there
is a kind of self-assertion which it is necessary for the wise to overcome.
Indignation is a submission of our thoughts, but not of our desires; the
Stoic freedom in which wisdom consists is found in the submission of our
desires, but not of our thoughts. From the submission of our desires springs
the virtue of resignation; from the freedom of our thoughts springs the whole
world of art and philosophy, and the vision of beauty by which, at last, we
half reconquer the reluctant world.

When first the opposition of fact and ideal grows fully visible, a spirit
of fiery revolt, of fierce hatred of the gods, seems necessary to the
assertion of freedom. To defy with Promethean constancy a hostile universe,
to keep its evil always in view, always actively hated, to refuse no pain
that the malice of Power can invent, appears to be the duty of all who will
not bow before the inevitable. But indignation is still a bondage, for it
compels our thoughts to be occupied with an evil world; and in the fierceness
of desire from which rebellion springs there is a kind of self-assertion
which it is necessary for the wise to overcome. Indignation is a submission
of our thoughts, but not of our desires; the Stoic freedom in which wisdom
consists is found in the submission of our desires, but not of our thoughts.
From the submission of our desires springs the virtue of resignation; from
the freedom of our thoughts springs the whole world of art and philosophy,
and the vision of beauty by which, at last, we half reconquer the reluctant
world.
...

We are of the sun and the moon and the stars.
The power manifested in the mind of God is projected

In [51]:
%%time
autocomplete(model=student.model, temp=1.0, n_ctx=128, n_generate=512)

=== Prompt ===
 the phantom eluded her.

"What is it?" she said again, studying him with knitted brows.

He bowed himself before her till he appeared to be no more than a bundle
of dirty linen.

"Let the gracious lady be warned by her servant," he said. "Fletcher
_sahib_ is a man of evil heart."

Beryl's eyes widened. Assuredly this was the last thing she had expected
to hear from such a source.

"What do you mean?" she
=== Autocompletion ===



RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

## plots

In [ ]:
import time
plot_data = {}
lag = 4096*4
X = Fun(Log2Sum(), student.times)
Y = Fun(TwoWindowFilter(lag=lag), student.grades)
Z = Fun(TwoWindowFilter(lag=lag), student.baseline_grades)
plot_data.update({f"grades": (X, Y)})
plot_data.update({f"baseline": (X, Z)})
Plot(**plot_data)

In [39]:
import time
plot_data_2 = {}
lag = 10
X = Fun(Count(), student.times)
Y = Fun(lambda x, y: x - y, student.grades, student.baseline_grades)
Y = Fun(TwoWindowFilter(lag=lag), Y.output, aux=Y)
plot_data_2.update({f"improvement": (X, Y)})
Plot(**plot_data_2)

## stats

In [54]:
import time

n = len(student.times)-1
t = time.time() - t_start
dn = n - n_of_last_baseline
dt = t - t_of_last_baseline

N = min(10000, (dn//2000)*1000)
if N == 0:
    N = dn // 2
y = np.mean(np.array(student.grades[n-N:n]))
y0 = np.mean(np.array(student.baseline_grades[n-N:n]))
dy = y - y0


message = '\n'.join([
    f"batch_size            = {student.batch_size}",
    f"example_length        = {student.example_length}",
    f"100*y                 = {int(y*1e6)/1e4}",
    f"n                     = {n} steps",
    f"t                     = {int(t)} seconds",
    f"n_of_last_baseline    = {n_of_last_baseline} steps",
    f"t_of_last_baseline    = {int(t_of_last_baseline)} seconds",
    f"steps per second      = {dn/dt}",
    f"y0                    = {int(y0*1e6)/1e6}",
    f"dy                    = {int(dy*1e6)/1e6}",
    f"dn                    = {dn}",
    f"dt                    = {dt}",
    f"dy/dn                 = {dy/dn}",
    f"dy/dt                 = {int(1e9 * dy/dt)}e-9 per second",
    f"time for 100%         = {(1.0)/(dy/dt)//36/100} hours",
])
print(message)

batch_size            = 1
example_length        = 129
100*y                 = 45.7881
n                     = 2179 steps
t                     = 23852 seconds
n_of_last_baseline    = 1793 steps
t_of_last_baseline    = 19534 seconds
steps per second      = 0.08940090700731945
y0                    = 0.451572
dy                    = 0.006309
dn                    = 386
dt                    = 4317.629573583603
dy/dn                 = 1.6344719235345175e-05
dy/dt                 = 1461e-9 per second
time for 100%         = 190.09 hours


In [37]:
if True:
    student.reset_baseline()
    n_of_last_baseline = len(student.times)-1
    t_of_last_baseline = time.time() - t_start

## save

In [ ]:
torch.save(student.model, f=path)

In [ ]:
import asyncio
async def autosave():
    while True:
        await asyncio.sleep(3600)
        torch.save(student.model, f='autosave.pt')
task = asyncio.create_task(autosave())


In [ ]:
chars_per_token = 

In [ ]:
lyles_constant = (9115131782/2)/14818489608 * log(50257)/log(256)
lyles_constant

In [ ]:
utf8grade = lambda x: 1 - (1 - x)*lyles_constant
grade = .7343
print(f"gpt2 grade = {grade}, utf8 grade = {utf8grade(grade)}, bpc = {(1-utf8grade(grade))*8}")

## parameter histograms

In [ ]:
histograms = []
for (idx, (pn, p)) in enumerate(student.model.named_parameters()):
    with torch.no_grad():
        print(idx, pn, torch.sqrt(torch.var(p)).item())
        Y, X = np.histogram(p.detach().cpu().numpy(), bins=int(sqrt(torch.numel(p))), density=True)
        print(X.shape, Y.shape)
        histograms.append(Plot(**{f"hist-{idx}": (X.tolist(), Y.tolist())}))

In [ ]:
histograms[4] # 3 7 9 13 15 21 43

## batch-level grade histogram

In [ ]:
Y, X = np.histogram(student.grades[-5000:], bins=256, range=(0,1.0), density=True)
V, U = np.histogram(student.baseline_grades[-5000:], bins=256, range=(0,1.0), density=True)
Plot(**{f"grade-hist": (X, Y), "baseline": (U, V)})

In [ ]:
model.n_ctx, model.d_model, model.d_hidden, model.n_layers

## example-level grade histogram

In [ ]:
def get_graded_examples():
    result = []
    for batch_idx in range(16):
        print(f"batch_idx = {batch_idx}/256")
        x = student.dataset.batch(student.batch_size, student.example_length)
        print(f"orig {x.shape}")
        with torch.no_grad():
            y = student.model(x)
            y = 1.0 - y.cpu().numpy()
            result.append(y)
    data = np.concatenate(result, axis=0)
    result = data.tolist()
    return result

In [ ]:
graded_examples = get_graded_examples()

In [ ]:
len(graded_examples)

In [ ]:
sum(x for x in graded_examples)/len(graded_examples)

In [ ]:
R = (0, 1)
def XYFor(k):
    es = graded_examples
    bins = int(sqrt(len(es)))
    Y, X = np.histogram(es, bins=bins, range=R, density=True)
    return (X, Y)
Plot(**{f"examples-hist-{k}": XYFor(k) for k in [1]})

In [ ]:
ord(' ')

In [ ]:
np.mean(example_grades)

In [ ]:
(1 - 0.7870894884999871)*8

In [ ]:
(1 - 0.8)*8

In [ ]:
(1 - 0.9)*8

In [ ]:
x = np.array([[1,2],[3,4]],dtype=np.uint8)

In [ ]:
np.unpackbits(x, axis=1)